In [1]:
import pandas as pd
from ucimlrepo import fetch_ucirepo

In [2]:
online_retail = fetch_ucirepo(id=352)
df = online_retail.data.original

df.dropna(subset=['CustomerID', 'Description'], inplace=True)
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']

# Dim Table: Customers
dim_customers = df[['CustomerID', 'Country']].drop_duplicates().reset_index(drop=True)

# Dim Table: Products
dim_products = df[['Description']].drop_duplicates().reset_index(drop=True)
dim_products['ProductID'] = dim_products.index + 1

# Mearge the ProductID
df = df.merge(dim_products, on='Description', how='left')

# Dim Table: Time
dim_time = df[['InvoiceDate']].drop_duplicates().reset_index(drop=True)
dim_time['TimeID'] = dim_time.index + 1
dim_time['Year'] = dim_time['InvoiceDate'].dt.year
dim_time['Month'] = dim_time['InvoiceDate'].dt.month
dim_time['Day'] = dim_time['InvoiceDate'].dt.day

# Mearge TimeID
df = df.merge(dim_time, on='InvoiceDate', how='left')

# Fact Table
fact_sales = df[['InvoiceNo', 'CustomerID', 'ProductID', 'TimeID', 'Quantity', 'UnitPrice', 'TotalPrice']].drop_duplicates()

In [3]:
# Save as a CSV files
dim_customers.to_csv('star_schema_tables/dim_customers.csv', index=False)
dim_products.to_csv('star_schema_tables/dim_products.csv', index=False)
dim_time.to_csv('star_schema_tables/dim_time.csv', index=False)
fact_sales.to_csv('star_schema_tables/fact_sales.csv', index=False)

print("Star schema created and CSV files saved.")

Star schema created and CSV files saved.


In [4]:
print("dim_customers:\n", dim_customers.head())
print("\n dim_products:\n", dim_products.head())
print("\n dim_time:\n", dim_time.head())
print("\n fact_sales:\n", fact_sales.head())

dim_customers:
    CustomerID         Country
0     17850.0  United Kingdom
1     13047.0  United Kingdom
2     12583.0          France
3     13748.0  United Kingdom
4     15100.0  United Kingdom

 dim_products:
                            Description  ProductID
0   WHITE HANGING HEART T-LIGHT HOLDER          1
1                  WHITE METAL LANTERN          2
2       CREAM CUPID HEARTS COAT HANGER          3
3  KNITTED UNION FLAG HOT WATER BOTTLE          4
4       RED WOOLLY HOTTIE WHITE HEART.          5

 dim_time:
           InvoiceDate  TimeID  Year  Month  Day
0 2010-12-01 08:26:00       1  2010     12    1
1 2010-12-01 08:28:00       2  2010     12    1
2 2010-12-01 08:34:00       3  2010     12    1
3 2010-12-01 08:35:00       4  2010     12    1
4 2010-12-01 08:45:00       5  2010     12    1

 fact_sales:
   InvoiceNo  CustomerID  ProductID  TimeID  Quantity  UnitPrice  TotalPrice
0    536365     17850.0          1       1         6       2.55       15.30
1    536365     178